In [53]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python /gpfsdswork/projects/idris/sos/ssos023/Projects/Plug-AI/Plug-AI/JZ_experiments/main.py \
--dataset_dir /gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/ \
--limit_sample 20 --batch_size 2 --lr 0.0001

Overwriting plug_ai.slurm


In [54]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 1503028


In [55]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1503028   gpu_p13  plug_ai  ssos023  R       1:32      1 r10i2n8
loading dataset...
got datalist, extract: 
 {'channel_0': '/gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/imagesTr/BRATS_001.nii.gz', 'label': '/gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/labelsTr/BRATS_001.nii.gz'}
dataset loaded ! Loading model...
Model loaded ! Initialize training...
start training loop, 10 steps per epoch
[Epoch 1/1 |  Step_Epoch 1/10 | Loss 2.5674943923950195]
[Epoch 1/1 |  Step_Epoch 2/10 | Loss 2.533360481262207]
[Epoch 1/1 |  Step_Epoch 3/10 | Loss 2.509921073913574]
[Epoch 1/1 |  Step_Epoch 4/10 | Loss 2.5486364364624023]
Interrupted by 2, shutting down


In [46]:
!scancel -u $USER -n plug_ai